In [2]:
# Computing and applying a linear minimum-norm inverse method on evoked/raw/epochs data

# Import necessary libraries and functions
from scipy.stats import zscore
import sklearn as sk
from scipy import signal
import copy  # This is a Python module that allows you to copy objects without changing the original object
from mne.preprocessing import ICA
from mne_icalabel import label_components
from sklearn.decomposition import PCA
from mne.minimum_norm import apply_inverse_epochs
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import sample, eegbci, fetch_fsaverage
import mne
import matplotlib.pyplot as plt
import os
import os.path as op
import numpy as np
import matplotlib
# matplotlib.use('Qt5Agg')  # Setting the backend BEFORE importing pyplot

# mne.viz.set_3d_backend("pyvista")


#################################################################################
# Adult template MRI (fsaverage)

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")


# Import necessary Python modules
matplotlib.use('Agg')  # disable plotting
mne.viz.set_browser_backend('matplotlib', verbose=None)
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')


# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')

subject_list = subject_list[-15:]

print(subject_list)
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}

for subject in subject_list:
    for mode in modes:
        print(subject, mode)
        # defining paths for current subject
        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'

        log_file = output_path+'log4.txt'
        log = open(log_file, "w")

        epochs = mne.read_epochs(output_path + subject+'_epoched.fif')

        # List of channels to drop
        channels_to_drop = ['LHEye', 'RHEye', 'Lneck', 'Rneck', 'RVEye', 'FPz']

        # Drop the channels from the epochs data
        try:
            epochs.drop_channels(channels_to_drop)
        except:
            pass

        # Adjust EEG electrode locations to match the fsaverage template, which are already in fsaverage's
        # # space (MNI space) for standard_1020
        montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
        montage = mne.channels.read_custom_montage(montage_path)
        epochs.set_montage(montage)
        # needed for inverse modeling
        epochs.set_eeg_reference(projection=True)

        # Compute the forward solution using the fsaverage template
        fwd = mne.make_forward_solution(
            epochs.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0, n_jobs=None
        )

        # Adjusting picks to EEG data
        picks = mne.pick_types(epochs.info, meg=False,
                               eeg=True, eog=True, stim=False)

        # Compute regularized noise covariance
        noise_cov = mne.compute_covariance(
            epochs, tmax=0.0, method=["shrunk", "empirical"], rank=None, verbose=True
        )

        fig_cov, fig_spectra = mne.viz.plot_cov(noise_cov, epochs.info)

        #################################################################################
        # Visualize the source space on the cortex

        # Read the forward solution
        mne.convert_forward_solution(fwd, surf_ori=True, copy=False)

        # Extract the source space information from the forward solution
        lh = fwd["src"][0]  # Visualize the left hemisphere
        verts = lh["rr"]  # The vertices of the source space
        tris = lh["tris"]  # Groups of three vertices that form triangles
        dip_pos = lh["rr"][lh["vertno"]]  # The position of the dipoles
        dip_ori = lh["nn"][lh["vertno"]]
        dip_len = len(dip_pos)
        dip_times = [0]

        # Create a Dipole instance
        actual_amp = np.ones(dip_len)  # misc amp to create Dipole instance
        actual_gof = np.ones(dip_len)  # misc GOF to create Dipole instance
        dipoles = mne.Dipole(dip_times, dip_pos,
                             actual_amp, dip_ori, actual_gof)
        trans = trans

        mne.write_forward_solution(
            output_path + '_forwardsolution_MRItemplate.fif', fwd, overwrite=True)
        #################################################################################
        # Inverse modeling: eLORETA on evoked data with dipole orientation discarded (pick_ori="None"), only magnitude kept

        # Create a loose-orientation inverse operator, with depth weighting
        inv = make_inverse_operator(
            epochs.info, fwd, noise_cov, fixed=False, loose=0.2, depth=0.8, verbose=True)

        # Compute eLORETA solution for each epoch
        snr = 3.0
        lambda2 = 1.0 / snr**2
        # pick_ori="None" --> Discard dipole orientation, only keep magnitude
        stcs = apply_inverse_epochs(
            epochs, inv, lambda2, "eLORETA", verbose=True, pick_ori=None)

        # Average the source estimates across epochs
        stc_avg = sum(stcs) / len(stcs)

        # Get the time of the peak magnitude
        _, time_max = stc_avg.get_peak(hemi="lh")

        # Visualization parameters
        kwargs = dict(
            hemi="lh",
            subjects_dir=subjects_dir,
            size=(600, 600),
            clim=dict(kind="percent", lims=[90, 95, 99]),
            smoothing_steps=7,
            time_unit="s",
            initial_time=time_max  # Set the initial_time to the time of the peak magnitude
        )

        # Visualizing the averaged source estimate with dipole magnitude
        # brain_magnitude = stc_avg.plot(**kwargs)
        # mne.viz.set_3d_view(figure=brain_magnitude, focalpoint=(0.0, 0.0, 50))

        # Average the data across all source space points
        avg_data = stc_avg.data.mean(axis=(0, 1))

        # Plot the average data as a function of time
        fig, ax = plt.subplots()
        print(stc_avg.times, avg_data)
        # ax.plot(1e3 * stc_avg.times, avg_data)
        # ax.set(xlabel="time (ms)", ylabel="eLORETA value (average)")
        # plt.show()

        print('Saving file')
        # Save the inverse solution data
        output_dir = output_path + 'stc/'  # Replace with your desired output directory
        for idx, stc in enumerate(stcs):
            inverse_solution_file = output_dir + f"{subject}_inversesolution_epoch{idx}.fif"
            print(inverse_solution_file)
            stc.save(inverse_solution_file, overwrite = True)

0 files missing from root.txt in /home/prakhar7/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage
Using matplotlib as 2D backend.
['401', '402', '403', '404', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416']
401 EC
Reading /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/401/EC/401_epoched.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
62 matching events found
No baseline correction applied
0 projection items activated
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Source space          : /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /home/prakhar7

/tmp/ipykernel_2770037/857062165.py:86: RuntimeWarning: This filename (../data/out/subjects/401/EC/401_epoched.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(output_path + subject+'_epoched.fif')
/tmp/ipykernel_2770037/857062165.py:101: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  epochs.set_montage(montage)
/tmp/ipykernel_2770037/857062165.py:101: RuntimeWarning: Not setting positions of 5 emg/eog channels found in montage:
['LHEye', 'RHEye', 'Lneck', 'Rneck', 'RVEye']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  epochs.set_montage(montage)


Source spaces are now in head coordinates.

Setting up the BEM model using /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif...

Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Employing the head->MRI coordinate transform with the BEM model.
BEM model fsaverage-5120-5120-5120-bem-sol.fif is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (will take a few...)
Checking surface interior status for 10242 points...
    Found  2433/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 7809 points outside using surface Qhull
    Found     0/ 7809 points outside using solid angles
    Total 10242/102

/tmp/ipykernel_2770037/857062165.py:143: RuntimeWarning: This filename (../data/out/subjects/401/EC/_forwardsolution_MRItemplate.fif) does not conform to MNE naming conventions. All forward files should end with -fwd.fif, -fwd.fif.gz, _fwd.fif or _fwd.fif.gz
  mne.write_forward_solution(
/tmp/ipykernel_2770037/857062165.py:143: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  mne.write_forward_solution(


Selected 58 channels
Creating the depth weighting matrix...
    58 EEG channels
    limit = 20485/20484 = 2.114152
    scale = 136601 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 6.3e-14 (2.2e-16 eps * 58 dim * 4.9  max singular value)
    Estimated rank (eeg): 57
    EEG: rank 57 computed from 58 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 7.16615
    scaling factor to adjust the trace = 4.46128e+23 (nchan = 58 nzero = 1)
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimens

...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Employing the head->MRI coordinate transform with the BEM model.
BEM model fsaverage-5120-5120-5120-bem-sol.fif is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (will take a few...)
Checking surface interior status for 10242 points...
    Found  2433/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 7809 points outside using surface Qhull
    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 8841.9 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points o

/tmp/ipykernel_2770037/857062165.py:143: RuntimeWarning: This filename (../data/out/subjects/416/EO/_forwardsolution_MRItemplate.fif) does not conform to MNE naming conventions. All forward files should end with -fwd.fif, -fwd.fif.gz, _fwd.fif or _fwd.fif.gz
  mne.write_forward_solution(
/tmp/ipykernel_2770037/857062165.py:143: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  mne.write_forward_solution(


Selected 58 channels
Creating the depth weighting matrix...
    58 EEG channels
    limit = 20485/20484 = 2.114152
    scale = 136601 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 1.9e-14 (2.2e-16 eps * 58 dim * 1.4  max singular value)
    Estimated rank (eeg): 57
    EEG: rank 57 computed from 58 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 4.08808
    scaling factor to adjust the trace = 4.56682e+23 (nchan = 58 nzero = 1)
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimens

Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch1.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch2.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch3.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch4.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch5.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch6.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch

Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch57.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch58.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch59.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch60.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch61.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
../data/out/subjects/416/EO/stc/416_inversesolution_epoch62.fif
Writing STC to disk...
Overwriting existing file.
Overwriting existing file.
[done]
